In [1]:
import pandas as pd
import os
import numpy as np
import datetime as dt

from os import listdir
from os.path import isfile, join
from os.path import getmtime

from zebra import remov_duplicates, human_format
from babel.numbers import format_number

In [2]:
idx = pd.IndexSlice

In [3]:
pd.set_option('display.max_rows', 100)

# Sources

## Parametriche

In [5]:
parametriche = r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Dashboard inflow\Parametriche\\"
onlyfiles = [f for f in listdir(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Dashboard inflow\Parametriche") if isfile(join(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Dashboard inflow\Parametriche", f))]

par = {}
for file in onlyfiles:
    file_name=file.split(".")[0]
    par[f"{file_name}"] = pd.read_excel(parametriche+file)

In [9]:
cod_prod = par["Classificazione_codice_prodotto"]
cod_prod["Codice PRODOTTO"] = cod_prod["Codice PRODOTTO"].apply(lambda x: str(x))
cod_prod = cod_prod.loc[:, ['Business area', 'MDM', 'GTM BDG', 'Solution',"Codice PRODOTTO"]]

sales_director = par["Sales_director"]
sales_director = sales_director.loc[:, ["Sales Director", "Area territoriale", "Rete"]]
sales_director["Sales Director"] = sales_director["Sales Director"].str.lower()

agente_ordine = par["Agente_ordine"]

esc_fatture = par["Esclusione_fatture"]
esc_fatture["Numero Fattura"] = esc_fatture["Numero Fattura"].astype(str)

esc_ordini = par["Esclusione_ordini"]
esc_ordini["Numero Ordine"] = esc_ordini["Numero Ordine"].astype(str)

li_prodotti = par["Linea_prodotti"]

## Inflow

In [7]:
inflow_prodotti = r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Analisi inflow\Inflow_prodotti\\"

onlyfiles = [f for f in listdir(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Analisi inflow\Inflow_prodotti") if isfile(join(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Analisi inflow\Inflow_prodotti", f))]

dfs = []
for file in onlyfiles:
    data = pd.read_excel(inflow_prodotti+file)
    dfs.append(data)
inflow = pd.concat(dfs, axis=0, ignore_index=True)
    
for col in inflow.select_dtypes(include=[object]).columns:
    inflow[col] = inflow[col].astype(str)

In [11]:
inflow.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 147882 entries, 0 to 147881
Data columns (total 21 columns):
 #   Column                           Non-Null Count   Dtype         
---  ------                           --------------   -----         
 0   Area (SW)                        147882 non-null  object        
 1   Prodotto (SW)                    147882 non-null  object        
 2   Soluzione (SW)                   147882 non-null  object        
 3   Codice PRODOTTO                  147882 non-null  object        
 4   Fornitura (SW)                   147882 non-null  object        
 5   Gruppo Prodotti Ricl             147882 non-null  object        
 6   Agente dell'ORDINE               147882 non-null  object        
 7   Sales Director                   147882 non-null  object        
 8   Canale di VENDITA                147882 non-null  object        
 9   Cliente Merce                    147882 non-null  object        
 10  Cliente FATTURA                  147882 non-

In [10]:
def creazione_dataframe(df_):
    
    df = inflow.merge(esc_fatture, on="Numero Fattura", how="left").merge(esc_ordini, on="Numero Ordine", how="left").merge(li_prodotti, on="Linea Prodotti", how="left").merge(cod_prod, how="left", on="Codice PRODOTTO")
    df.rename(columns={"Data Ins. Ordine (monitoraggio)":"Data"}, inplace=True)
    df = df.loc[
        (df["Canale di VENDITA"] != "E-Commerce")&(df["Escludi fatture"].isna())&(df["Escludi ordine"].isna()),
        ["Data", "Linea", "MDM", "GTM BDG", "Business area", "Raccolto"]
        ]
    return df

deck = inflow.pipe(creazione_dataframe)

In [11]:
deck_ytd = deck.groupby([deck["Data"].dt.year, "Linea", "MDM", "GTM BDG", "Business area"]).agg({"Raccolto":"sum"}).unstack(0)
deck_ytd.columns = deck_ytd.columns.droplevel(level=0)
deck_ytd.style.format(decimal=",", thousands=".", precision=0)

In [12]:
deck_gen = deck.groupby([deck["Data"].dt.to_period('M'), "Linea", "MDM", "GTM BDG", "Business area"]).agg({"Raccolto":"sum"}).unstack(0).loc[:, idx["Raccolto", ["2022-01", "2023-01"]]]
deck_gen.columns = deck_gen.columns.droplevel(level=0)

deck_gen.style.format(decimal=",", thousands=".", precision=0)

In [20]:
registro_ordini = r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Dashboard inflow\Dashboard inflow canali e prodotti\Speciali\Dashboard inflow - Deck prodotti\Deck prodotti per BR 2.xlsx"
with pd.ExcelWriter(registro_ordini) as writer:
    deck_ytd.to_excel(writer, sheet_name="Trend YTD")
    deck_gen.to_excel(writer, sheet_name="Confronto gennaio")